In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
from PIL import Image
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.1.27 🚀 Python-3.12.1 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Setup complete ✅ (16 CPUs, 31.9 GB RAM, 729.6/930.4 GB disk)


Split raw dataset into YOLO compatible form

In [5]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image

# Define class labels and their corresponding IDs
class_labels = ['aerial', 'artillery', 'bmp', 'BTR', 'infantry', 'LAV', 'MLRS', 'tank']
class_map = {label: idx for idx, label in enumerate(class_labels)}

# Load the dataset
csv_file_path = './dataset/AVs/labels.csv'  # Update this path if necessary
images_dir = './dataset/AVs/images'
df = pd.read_csv(csv_file_path)

# Map class labels to IDs
df['label'] = df['label'].map(class_map)

# Splitting the dataset
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

# Create directories for the YOLO dataset
dataset_base = './dataset/AVs/yolo_dataset'
os.makedirs(os.path.join(dataset_base, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(dataset_base, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(dataset_base, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(dataset_base, 'labels/val'), exist_ok=True)

# Function to convert and save annotations
def save_annotations(row, images_dir, labels_dir, is_train):
    # Calculate YOLO format bounding box
    image_path = os.path.join(images_dir, row['image'])
    image_width, image_height = Image.open(image_path).size
    x_center = ((row['xmin'] + row['xmax']) / 2) / image_width
    y_center = ((row['ymin'] + row['ymax']) / 2) / image_height
    bbox_width = (row['xmax'] - row['xmin']) / image_width
    bbox_height = (row['ymax'] - row['ymin']) / image_height
    
    # Write annotation in YOLO format
    annotation = f"{row['label']} {x_center} {y_center} {bbox_width} {bbox_height}\n"
    base_filename = os.path.splitext(row['image'])[0]
    annotation_file_path = os.path.join(labels_dir, base_filename + '.txt')
    
    with open(annotation_file_path, 'w') as file:
        file.write(annotation)
    
    # Copy image to appropriate directory
    dst_image_path = os.path.join(dataset_base, 'images', 'train' if is_train else 'val', row['image'])
    shutil.copy(image_path, dst_image_path)

# Process and save annotations
for _, row in train_df.iterrows():
    save_annotations(row, images_dir, os.path.join(dataset_base, 'labels/train'), True)

for _, row in val_df.iterrows():
    save_annotations(row, images_dir, os.path.join(dataset_base, 'labels/val'), False)

print("YOLO dataset creation complete.")


YOLO dataset creation complete.


In [2]:
# Initialize the model with the pre-trained weights
model = YOLO('yolov8s.pt')

# Train the model on your dataset
model.train(data='C:\\Users\\Jack Bowman\\Documents\\Programs\\PytScripts\\CV\\YOLO_video\\AV_dataset.yaml', epochs=50, batch=-1)

# Evaluate the model on the validation set
results = model.val(data='C:\\Users\\Jack Bowman\\Documents\\Programs\\PytScripts\\CV\\YOLO_video\\AV_dataset.yaml')
print("Results on Validation Set:", results)


Ultralytics YOLOv8.1.27 🚀 Python-3.12.1 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\Jack Bowman\Documents\Programs\PytScripts\CV\YOLO_video\AV_dataset.yaml, epochs=50, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_

train: Scanning C:\Users\Jack Bowman\Documents\Programs\PytScripts\CV\YOLO_video\dataset\AVs\yolo_dataset\labels\train.cache... 2059 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2059/2059 [00:00<?, ?it/s]
val: Scanning C:\Users\Jack Bowman\Documents\Programs\PytScripts\CV\YOLO_video\dataset\AVs\yolo_dataset\labels\val.cache... 915 images, 0 backgrounds, 0 corrupt: 100%|██████████| 915/915 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      7.39G      1.293       3.62      1.589         24        640: 100%|██████████| 65/65 [00:28<00:00,  2.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.04it/s]

                   all        915        915      0.411      0.563      0.468      0.238


...


50 epochs completed in 0.482 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 22.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.12.1 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 168 layers, 11128680 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:05<00:00,  2.57it/s]


                   all        915        915      0.728      0.686      0.746      0.528
                aerial        915         81       0.78      0.741      0.797      0.439
             artillery        915         75      0.643      0.573        0.6      0.382
                   bmp        915        168      0.802      0.869      0.912      0.677
                   BTR        915         83      0.866      0.781      0.881      0.722
              infantry        915        127      0.371      0.173      0.216      0.119
                   LAV        915        101      0.734      0.723      0.813      0.653
                  MLRS        915        111      0.837      0.833      0.884      0.631
                  tank        915        169      0.793      0.799      0.863      0.605
Speed: 0.1ms preprocess, 3.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\train
Ultralytics YOLOv8.1.27 🚀 Python-3.12.1 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce R

val: Scanning C:\Users\Jack Bowman\Documents\Programs\PytScripts\CV\YOLO_video\dataset\AVs\yolo_dataset\labels\val.cache... 915 images, 0 backgrounds, 0 corrupt: 100%|██████████| 915/915 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.66it/s]


                   all        915        915      0.727      0.687      0.746      0.529
                aerial        915         81       0.77      0.741      0.791      0.434
             artillery        915         75      0.643      0.573        0.6      0.382
                   bmp        915        168      0.803      0.881      0.914      0.675
                   BTR        915         83      0.867      0.782      0.883      0.728
              infantry        915        127      0.366      0.165      0.215      0.119
                   LAV        915        101      0.742      0.723      0.814      0.655
                  MLRS        915        111      0.837      0.833      0.888      0.633
                  tank        915        169      0.793      0.799      0.866      0.605
Speed: 0.1ms preprocess, 5.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\train2
Results on Validation Set: ultralytics.utils.metrics.DetMetrics object with attri

In [3]:
model.save('trained_yolov8s.pt')

In [4]:

results = model.predict(source="0",show=True)
print(results)


1/1: 0... Success ✅ (inf frames of shape 640x480 at 30.00 FPS)


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 (no detections), 526.0ms
0: 480x640 (no detections), 6.0ms
0: 480x640 (no detections), 6.0ms
0: 480x640 (no detections), 6.0ms
0: 480x640 (no detections), 8.0ms
0: 480x640 (no detections), 7.0ms
0: 480x640 (no detections), 6.0ms
0: 480x640 (no detections), 6.0ms
0: 480x640 1 aerial, 6.0ms
0: 480x640 1 aerial, 6.0ms
0: 480x640 1 aerial, 7.0ms
0: 480x640 (no detections), 7.0ms
0: 

KeyboardInterrupt: 

: 